## Linear Layers

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Identity

In [2]:
"""
nn.Identity(): (*) -> (*)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.Identity.html
"""Define Layer"""
identity = nn.Identity()

"""Forward Pass"""

x = torch.randn(2, 3)
y = identity(x)

assert y.shape == x.shape
assert torch.equal(y, x)

### Linear

In [3]:
"""
nn.Linear(
    in_features=Hin,
    out_features=Hout,
):  (*, Hin)
->  (*, Hout)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html
"""Define Layer"""
"""1. Position Arguments"""
in_features, out_features, bias = 2, 3, True

linear = nn.Linear(
    in_features,  # Hin, Required
    out_features,  # Hout, Required
    bias,  # default=True
    device=None,
    dtype=None,
)

"""Forward Pass"""

batch_sizes = (2, 3, 4)

x = torch.randn(*batch_sizes, in_features)  # (N, Hin)
y = linear(x)

assert y.shape == (*batch_sizes, out_features)  # (N, Hout)
assert torch.allclose(y, F.linear(x, linear.weight, linear.bias))
assert torch.allclose(
    F.linear(x, linear.weight, linear.bias),
    x @ linear.weight.t() + (linear.bias if bias else 0),
)

### Bilinear

In [4]:
"""
nn.Bilinear(
    in1_features=Hin1,
    in2_features=Hin2,
    out_features=Hout,
):  (*, Hin1), (*, Hin2)
->  (*, Hout)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.Bilinear.html
"""Define Layer"""
"""1. Position Arguments"""
in1_features, in2_features, out_features, bias = 2, 3, 4, True

bilinear = nn.Bilinear(
    in1_features,  # Hin1, Required
    in2_features,  # Hin2, Required
    out_features,  # Hout, Required
    bias,
    device=None,
    dtype=None,
)

"""Forward Pass"""
batch_sizes = (2, 3, 4)

x1 = torch.randn(*batch_sizes, in1_features)  # (N, Hin1)
x2 = torch.randn(*batch_sizes, in2_features)  # (N, Hin2)
y = bilinear(x1, x2)

assert y.shape == (*batch_sizes, out_features)  # (N, Hout)
assert torch.allclose(
    y,
    F.bilinear(x1, x2, bilinear.weight, bilinear.bias),
)
assert torch.allclose(
    y,
    torch.einsum("...i,kij,...j->...k", x1, bilinear.weight, x2)
    + (bilinear.bias if bias else 0),
    atol=1e-6,
)

### LazyLinear

In [5]:
"""
nn.LazyLinear(
    out_features=Hout
):  (*, Hin)
->  (*, Hout)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.LazyLinear.html

"""Define Layer"""
"""1. Position Arguments"""
out_features, bias = 2, True

lazy_linear = nn.LazyLinear(
    out_features,  # Hout
    bias,
    device=None,
    dtype=None,
)

"""Forward Pass"""

batch_size, in_features = (3, 4), 5

x = torch.randn(*batch_size, in_features)  # (N, Hin)
y = lazy_linear(x)

assert y.shape == (*batch_size, out_features)  # (N, Hout)
assert torch.allclose(y, F.linear(x, lazy_linear.weight, lazy_linear.bias), atol=1e-6)